In [1]:
#import os
#import geopandas as gpd
import pandas as pd
import numpy as np
#import requests
#import json
#import zipfile
import datetime
import time
#from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import shape
import utils
import importlib
from sklearn.mixture import BayesianGaussianMixture
import nimfa
import scipy
from scipy import stats
import itertools


In [53]:
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\hildlars\\OneDrive - Universitetet i Oslo\\NordicMathCovid\\GitProjects\\NordicMobilityData_WorkingRepo\\PythonCode\\GaussianMixture\\utils.py'>

In [2]:
FolderPath = '../Data/FinSwe/' # Path to raw data files for SWE-FIN sensors.

Sensors = ["1433","1432","1435","1436","1431"]

Read data for each sensor, from .csv files downloaded using the function 'get_traffic_FIN()' in the file road_traffic_data_methodologyPaper.ipynb

In [3]:


sensor_dict = {}
all_data = pd.DataFrame()
for s in Sensors:
    sensor_data = pd.read_csv(FolderPath+s+'_by_length_minute.csv', sep=',',usecols= ['TMS point id', 'year', 'days', 'hour','minute', 'v_type', 'direction', 'total_vehicles', 'date'], parse_dates=['date'])

    all_data = pd.concat([all_data,sensor_data])


In [4]:
WhichVehicles = 'Small'
data = all_data.copy()
min_date = datetime.datetime(2017, 1, 1)
max_date = datetime.datetime(2023, 12, 31)

In [5]:
def agg_APIdata_SWEFIN(data,WhichVehicles,min_date = datetime.datetime(2017, 1, 1)  , max_date = datetime.datetime(2023, 5, 22)):

#WhichVehicles can be 'Small', 'Heavy', 'Total' or 'Both' 

    agg_data = data[['TMS point id','hour','minute', 'v_type', 'direction', 'total_vehicles', 'date']].reset_index(drop = True).copy()
    
    # What vehicle lengths do we want
    if WhichVehicles == 'Total':
        agg_data = agg_data.groupby(['TMS point id','hour','minute','direction','date'])['total_vehicles'].sum().reset_index()
    elif WhichVehicles == 'Small':
        agg_data = agg_data[agg_data['v_type']=='<5.6m'].drop(columns='v_type').reset_index()
    elif WhichVehicles == 'Heavy':
        agg_data = agg_data[agg_data['v_type']=='>=5.6m'].drop(columns='v_type').reset_index()




    agg_data = agg_data.rename({'TMS point id': 'sensor_id',
                                'direction': 'sensor_dir'},axis=1)
    


    agg_data['dest_country'] = np.where(agg_data['sensor_dir'] == 2, 'FIN','SWE')
    agg_data['origin_country'] = np.where(agg_data['sensor_dir'] == 2, 'SWE','FIN')

    ##Create a new variables that contain both sensor name and country name
    agg_data["sensor_id"] = agg_data["sensor_id"].astype(str)
    agg_data['sensor_origin'] = agg_data[['sensor_id', 'origin_country']].agg(', '.join, axis=1)
    agg_data['sensor_destination'] = agg_data[['sensor_id', 'dest_country']].agg(', '.join, axis=1)


    agg_data["minute"] = pd.to_timedelta(agg_data["minute"], unit="min")
    agg_data["hour"] = pd.to_timedelta(agg_data["hour"], unit="h")


    agg_data['date'] = agg_data['date'] + agg_data['hour'] + agg_data['minute']
    agg_data = agg_data[(agg_data.date > min_date) & (agg_data.date < max_date)].copy().reset_index()
    

    min_date = agg_data.date.min()
    max_date = agg_data.date.max()

    if WhichVehicles == 'Both':
        agg_data = agg_data[['sensor_origin','sensor_destination','date','v_type','total_vehicles']].copy()

    else:
        agg_data = agg_data.groupby(['sensor_origin', 'sensor_destination', 'date'])['total_vehicles'].sum().reset_index().copy()
        rm_idx = np.where(np.isnan(agg_data.total_vehicles))
        agg_data = agg_data.drop(index = rm_idx[0])
        agg_data['total_vehicles'] = agg_data['total_vehicles'].apply(lambda x: int(x))

        f = lambda x: x.reindex(pd.date_range(min_date,
                                                max_date,
                                                name='date',
                                                freq='1min'), fill_value=0)

    #return(data)

        agg_data = (agg_data.set_index('date')
                    .groupby(["sensor_origin", "sensor_destination"])["total_vehicles"]
                    .apply(f)
                    .reset_index())
        agg_data = agg_data.pivot_table(index=["sensor_origin","sensor_destination"], columns=["date"],values=["total_vehicles"] ).droplevel(level = 0,axis = 1 )

    return(agg_data)

In [6]:
# For 2017-2023 this takes about 4 min to run
agg_data = agg_APIdata_SWEFIN(all_data, 'Small',min_date = datetime.datetime(2017, 1, 1)  , max_date = datetime.datetime(2023, 12, 31))

In [7]:
# This takes about 18 min to run
models_pre, data_pre = utils.fit_period(agg_data, d1=datetime.date(2017,1,1), d2=datetime.date(2020,3,1), hourly = False, nSamp = 10000, Normalize = False, N = 10, seed=1923, FitMethod = 'Bayesian')
# Save the fitted model and data
models_pre.to_pickle('../Data/models_fin.pkl')
data_pre.to_pickle('../Data/data_fin.pkl')
agg_data.to_pickle('../Data/agg_data_fin.pkl')

c:\Users\hildlars\OneDrive - Universitetet i Oslo\NordicMathCovid\PanNordicMobility\FinalFiles_commutingPaper\Code\utils.py:122: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  models = data[c].reset_index("weekday").groupby("weekday").agg(lambda x : fit_model(x,hourly_data, nSamp, N = N, hourminute = True,seed = seed, FitMethod = FitMethod))
